In [4]:
import pyodbc
import pandas as pd
server = 'romcapurro.database.windows.net'
database = 'telecom'
username = 'romcapurro'
password = '{C!rc4C0mb4t}'
driver= '{ODBC Driver 18 for SQL Server}'

# Crear un DataFrame de ejemplo
df = pd.read_csv('Internet_Penetracion_cada100.csv')

# Cadena de conexión
conn_str = f'DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30'

# Conexion
conn = pyodbc.connect(conn_str)

# Crear un cursor
cursor = conn.cursor()

# Nombre de la tabla en Azure SQL Database
table_name = 'internet_accesos_100_hogares'

# Eliminar la tabla si ya existe
cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};")

# Crear la tabla
cursor.execute(f'''
    CREATE TABLE {table_name} (
        anio INT,
        trimestre INT,
        provincia VARCHAR(100),
        accesos_por_cada_100_hogares VARCHAR(10)
    );
''')

conn.commit()

# Insertar datos en la tabla
for index, row in df.iterrows():
    cursor.execute(f"INSERT INTO {table_name} (anio, trimestre, provincia, accesos_por_cada_100_hogares) VALUES ({row['Año']}, {row['Trimestre']}, '{row['Provincia']}', '{row['Accesos por cada 100 hogares'].replace('"', "")}')")

# Confirmar la transacción
conn.commit()

# Cerrar la conexión
conn.close()